# Using Genestack Omics APIs for eQTL analysis

### 1. Connect to the instance

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os, time
import integration_curator # Genestack client library

os.environ['PRED_SPOT_HOST'] = 'occam.genestack.com'
os.environ['PRED_SPOT_TOKEN'] = '<your token>'
os.environ['PRED_SPOT_VERSION'] = 'default-released'

omics_api = integration_curator.OmicsQueriesApi()

### 2. Get samples

In [2]:
study_filter = 'genestack:accession=GSF535886'
sample_field = 'Species Or Strain'
sample_filter = ' OR '.join(['"%s"="%s"' % (sample_field, x) 
                             for x in ['British', 'Finnish']])

start = time.time()
data = omics_api.search_samples(
    study_filter=study_filter, 
    sample_filter=sample_filter
)
samples = pd.DataFrame.from_dict([item['metadata'] for item in data.data])
print('Time to get %s samples: %i seconds\n' % (samples.shape[0], time.time()-start))

samples.head()

Time to get 182 samples: 0 seconds



,genestack:accession,Sample Source ID,Sample Name,Organism,Disease,Tissue,Cell Type,Cell Line,Sampling Site,Age,...,Compound Dose,Compound Dose Unit,Raw Data Files,Processed Data Files,Import Source URL,Sex,Health,INDIVIDUALNAME,Sample Source,Species Or Strain
0,GSF535900,HG00111,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,Healthy,HG00111,IGSR,British
1,GSF535899,HG00110,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,Healthy,HG00110,IGSR,British
2,GSF535902,HG00113,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,Healthy,HG00113,IGSR,British
3,GSF535901,HG00112,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,Healthy,HG00112,IGSR,British
4,GSF535896,HG00106,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,Healthy,HG00106,IGSR,British


### 3. Get and compare genotypes across groups

In [3]:
vx_query = 'Intervals=4:142142600-142143000'
# vx_query = 'Gene=ENSG00000109445'

start = time.time()
data = omics_api.search_variant_data(
    study_filter=study_filter,
    sample_filter=sample_filter,
    vx_query=vx_query,
    page_limit=20000
)
def normalise_genotype(gt): return '1|0' if gt == '0|1' else gt
genotypes = pd.DataFrame.from_dict({'genestack:accession': x['relationships']['sample'], 
                                    'Genotype': normalise_genotype(x['genotype']['GT']),
                                    'Location': '%s:%s' % (x['contig'],x['start']),
                                    'ID': ', '.join(x['variationId']),
                                    'Ref / Alt': x['reference'] + ' / ' + ', '.join(x['alteration'])
                                   } 
                                   for x in data.data)

print('Time to get %s genotypes: %i seconds\n' % (genotypes.shape[0], time.time()-start))

samples_genotypes = pd.merge(samples, genotypes)
def f(x):
    d = {}
    for group in x[sample_field]:
        genotypes = x.loc[x[sample_field] == group, 'Genotype']
        genotypes = '|'.join(genotypes).split('|')
        ac = sum([gt == '1' for gt in genotypes])
        an = ac + sum([gt == '0' for gt in genotypes])
        af = round(ac/an, 2)
        d[group+' AF (AC/AN)'] = '%s (%s/%s)' % (af, ac, an)
    return pd.Series(d)
    
samples_genotypes.groupby(['ID', 'Location', 'Ref / Alt']).apply(f)

Time to get 724 genotypes: 1 seconds



,,,British AF (AC/AN),Finnish AF (AC/AN)
ID,Location,Ref / Alt,,
rs10023024,4:142142687,A / G,0.0 (0/176),0.0 (0/186)
rs146362755,4:142142703,A / G,0.01 (2/176),0.0 (0/186)
rs148317497,4:142142719,C / T,0.0 (0/176),0.0 (0/186)
rs17007017,4:142142729,A / G,0.26 (46/176),0.19 (36/186)


### 4. Get and compare expression values across groups

In [4]:
gene = 'ENSG00000109445'
ex_query = 'Gene=%s MinValue=0.0' % gene

start = time.time()
data = omics_api.search_expression_data(
    study_filter=study_filter,
    sample_filter=sample_filter,
    ex_query=ex_query
)
expressions = pd.DataFrame.from_dict({'genestack:accession': item['relationships']['sample'], 
                                      'expression': item['expression'],
                                     'Gene': item['gene']} for item in data.data)
print('Time to get %s expression values: %i seconds\n' % (expressions.shape[0], time.time()-start))

samples_expressions = pd.merge(samples, expressions)
def f(x):
    d = {}
    for group in x[sample_field]:
        exprs = x.loc[x[sample_field] == group, 'expression']
        quartiles = exprs.quantile([.25, .5, .75])
        d[group+' Median Expression (Q1, Q3)'] = '%i (%i, %i)' % (quartiles.iloc[0], quartiles.iloc[1], quartiles.iloc[2])
    return pd.Series(d)
    
samples_expressions.groupby(['Gene']).apply(f)

Time to get 174 expression values: 0 seconds



,"British Median Expression (Q1, Q3)","Finnish Median Expression (Q1, Q3)"
Gene,,
ENSG00000109445,"45 (50, 55)","46 (51, 56)"
